In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [33]:
df=pd.read_csv("/kaggle/input/car-evaluation-classification/cars.csv")
df=pd.get_dummies(df).astype(float)

In [34]:
df.isnull().sum()

buying_high       0
buying_low        0
buying_med        0
buying_vhigh      0
maint_high        0
maint_low         0
maint_med         0
maint_vhigh       0
doors_2           0
doors_3           0
doors_4           0
doors_5more       0
persons_2         0
persons_4         0
persons_more      0
lug_boot_big      0
lug_boot_med      0
lug_boot_small    0
safety_high       0
safety_low        0
safety_med        0
class_acc         0
class_good        0
class_unacc       0
class_vgood       0
dtype: int64

In [35]:
df["persons_more"]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
1723    1.0
1724    1.0
1725    1.0
1726    1.0
1727    1.0
Name: persons_more, Length: 1728, dtype: float64

In [89]:
X=df.drop(columns=["persons_more"])
Y=df["persons_more"]


In [90]:
X.shape

(1728, 24)

In [91]:
Y.shape

(1728,)

In [92]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [93]:
X_train.shape

(1382, 24)

In [94]:
Y_train.shape

(1382,)

In [38]:
random_classif=RandomForestClassifier()
random_classif.fit(X_train,Y_train)
random_classif.score(X_test,Y_test)

1.0

In [39]:
log_classif=LogisticRegression()
log_classif.fit(X_train,Y_train)
log_classif.score(X_test,Y_test)

1.0

In [40]:
!pip install cirq

In [41]:
import cirq

In [86]:
X_train.shape

(10, 6)

In [88]:
Y_train.shape

(1382,)

In [111]:
from keras.layers import Layer,Dense,Flatten,Input
from keras.models import Sequential
import tensorflow as tf

In [121]:
def quantum_circuit(data):
    qubit=[cirq.GridQubit(0,i) for i in range(4)]
    circuit=cirq.Circuit()
#     da=data[:4,:4]
    for i,value in enumerate(data):
        circuit.append(cirq.rx(value*2*np.pi).on(qubit[i%len(qubit)]))
        circuit.append(cirq.H(qubit[i%len(qubit)]))
        circuit.append(cirq.ry(value*2*np.pi).on(qubit[i%len(qubit)]))
        circuit.append(cirq.X(qubit[i%len(qubit)]))
        circuit.append(cirq.rz(value*2*np.pi).on(qubit[i%len(qubit)]))
        circuit.append(cirq.Z(qubit[i%len(qubit)]))
        if i+1<len(qubit):
            circuit.append(cirq.CNOT(qubit[i%len(qubit)],qubit[(i+1)%len(qubit)]))
        circuit.append(cirq.measure(qubit[i%len(qubit)],key=f"qubit_{i}"))
    return circuit,qubit
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
        circuit,qubit=quantum_circuit(inputs_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(qubit)):
            measure0.append(result.measurements[f"qubit_{i}"].flatten()[0])
        results.append(measure0)
    return np.array(results,dtype=np.float32)
# X_train = np.random.rand(10, 6)  # Example data
# da2 = run_quantum_circuit(X_train)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        output=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        output.set_shape((inputs.shape[0],inputs.shape[1]))
        return output
model=Sequential([
#     Input(shape=(4,)),

#     Flatten(),
    QuantumLayer(),
    Dense(32,activation="relu"),
    Dense(64,activation="relu"),
    Dense(128,activation="relu"),

#     Flatten(),
    Dense(4,activation="softmax")
    
])
model.compile(optimizer='adam',loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.fit(X_train.iloc[:4,:4],Y_train[:4],epochs=5)

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 1.2887
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7500 - loss: 1.2764
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.7500 - loss: 1.2591
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2500 - loss: 1.2605
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 1.0000 - loss: 1.1906


In [102]:
output = QuantumLayer()(X_train.iloc[:4,:4])  # Just a sample
print(output.shape)

(4, 4)


In [53]:
X_test.dtypes

buying_high       float64
buying_low        float64
buying_med        float64
buying_vhigh      float64
maint_high        float64
maint_low         float64
maint_med         float64
maint_vhigh       float64
doors_2           float64
doors_3           float64
doors_4           float64
doors_5more       float64
persons_2         float64
persons_4         float64
lug_boot_big      float64
lug_boot_med      float64
lug_boot_small    float64
safety_high       float64
safety_low        float64
safety_med        float64
class_acc         float64
class_good        float64
class_unacc       float64
class_vgood       float64
dtype: object

In [51]:
da23 = run_quantum_circuit(X_test)

TypeError: can't multiply sequence by non-int of type 'float'